In [4]:
import os
import pickle
import mediapipe as mp
import cv2

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = './data'
data = []
labels = []

for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []
        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                hand_data = []
                x_hand = []
                y_hand = []
                
                # Collect landmark data for each hand separately
                for lm in hand_landmarks.landmark:
                    x_hand.append(lm.x)
                    y_hand.append(lm.y)
                
                for lm in hand_landmarks.landmark:
                    hand_data.append(lm.x - min(x_hand))
                    hand_data.append(lm.y - min(y_hand))
                
                # Append hand data for both hands
                data_aux.extend(hand_data)
        
        if data_aux:  # Check if landmarks were extracted
            data.append(data_aux)
            labels.append(dir_)

# Save the dataset
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)
